In [1]:
import deep_rl

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.nn import functional as F
from torch.autograd import Variable
from torch import nn, optim
import torch.utils.data

# load as dask array
import dask.array as da
import dask
import h5py

import os
import glob
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

/home/wassname/.pyenv/versions/3.5.3/envs/jupyter3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from world_models_sonic.models.vae import VAE6, loss_function_vae
from world_models_sonic.helpers.summarize import TorchSummarizeDf
from world_models_sonic.helpers.dataset import load_cache_data
from world_models_sonic.models.rnn import MDNRNN2
from world_models_sonic.models.inverse_model import InverseModel
from world_models_sonic.models.world_model import WorldModel
from world_models_sonic import config

In [5]:
"""
In jupyter notebook simple logging to console
"""
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Init

In [6]:
cuda= torch.cuda.is_available()
env_name='sonic256'
num_epochs=200
batch_size = 2

# VAE loss function
lambda_vae_kld = 0.25
C = 0
z_dim = 256 # latent dimensions

# RNN
action_dim = 10
seq_len = 5
image_size=256
chunksize=seq_len*20



# loss function weights
lambda_vae = 1/100
lambda_finv = 1

data_cache_file = os.path.join(config.base_vae_data_dir, 'sonic_rnn_256_v30.hdf5')
NAME='RNN_v3b_256im_512z_1512_v4_greenfield'

# Load Data

# Load VAE

In [7]:
# Load VAE
# TODO swap z and k dim, since it's inconsistent with other models
vae = VAE6(image_size=image_size, z_dim=32, conv_dim=48, code_dim=8, k_dim=z_dim)
if cuda:
    vae.cuda()
    
# # Resume

# checkpoint_file = './outputs/models/VAE6_6_256im_512z_inception_CVAE_greenfields_state_dict.pkl'
# if os.path.isfile(checkpoint_file):
#     state_dict = torch.load(checkpoint_file)
#     vae.load_state_dict(state_dict)
#     print('loaded checkpoint_file {checkpoint_file}'.format(checkpoint_file=checkpoint_file))
    
save_file = './outputs/models/{NAME}-vae_state_dict.pkl'.format(NAME=NAME)
if os.path.isfile(save_file):
    state_dict = torch.load(save_file)
    vae.load_state_dict(state_dict)
    print('loaded save_file {save_file}'.format(save_file=save_file))

loaded save_file ./outputs/models/RNN_v3b_256im_512z_1512_v4_greenfield-vae_state_dict.pkl


# Load RNN

In [8]:
# Load MDRNN
action_dim, hidden_size, n_mixture, temp = action_dim, 128, 3, 0.0


mdnrnn = MDNRNN2(z_dim, action_dim, hidden_size, n_mixture, temp)

if cuda:
    mdnrnn = mdnrnn.cuda()

In [9]:
# # Resume?
save_file = './outputs/models/{NAME}-mdnrnn_state_dict.pkl'.format(NAME=NAME)
if os.path.isfile(save_file):
    state_dict = torch.load(save_file)
    mdnrnn.load_state_dict(state_dict)
    print('loaded {save_file}'.format(save_file=save_file))

loaded ./outputs/models/RNN_v3b_256im_512z_1512_v4_greenfield-mdnrnn_state_dict.pkl


# FInverse Model

In [10]:
finv = InverseModel(z_dim, action_dim, hidden_size=256).cuda()

In [11]:
# # Resume?
save_file = './outputs/models/{NAME}-finv_state_dict.pkl'.format(NAME=NAME)
if os.path.isfile(save_file):
    state_dict = torch.load(save_file)
    finv.load_state_dict(state_dict)
    print('loaded {save_file}'.format(save_file=save_file))

loaded ./outputs/models/RNN_v3b_256im_512z_1512_v4_greenfield-finv_state_dict.pkl


# Init

In [12]:
        
world_model = WorldModel(vae, mdnrnn, finv)
world_model.eval()
world_model

WorldModel(
  (vae): VAE6(
    (logvar): Linear(in_features=2048, out_features=256, bias=True)
    (mu): Linear(in_features=2048, out_features=256, bias=True)
    (z): Linear(in_features=256, out_features=2048, bias=True)
    (encoder): Sequential(
      (0): BasicConv2d(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): LeakyReLU(negative_slope=0.01, inplace)
      )
      (1): ConvBlock5(
        (conv0): InceptionA(
          (branch1x1): BasicConv2d(
            (conv): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): LeakyReLU(negative_slope=0.01, inplace)
          )
          (branch5x5_1): BasicConv2d(
            (conv): Conv2d(48, 9, kernel_size=(1, 1), stride=(1, 1))
            (bn): BatchNorm2d(9, eps=1e-05, momentum=0.

In [13]:

# import torch.optim.lr_scheduler
# optimizer = optim.Adam(world_model.parameters(), lr=1e-3)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)
# optimizer

# summarize

In [14]:
img = np.random.randn(image_size, image_size, 3)
action = np.array(np.random.randint(0,action_dim))[np.newaxis]
action = Variable(torch.from_numpy(action)).float().cuda()[np.newaxis]
gpu_img = Variable(torch.from_numpy(img[np.newaxis].transpose(0, 3, 1, 2))).float().cuda()
if cuda:
    gpu_img = gpu_img.cuda()
with TorchSummarizeDf(vae) as tdf:
    x, mu_vae, logvar_vae = vae.forward(gpu_img)
    z = vae.sample(mu_vae, logvar_vae)
    df_vae = tdf.make_df()
#     loss_recon, loss_KLD = loss_function_vae(Y, x, mu_vae, sigma_vae)
#     loss_vae = loss_recon + lambda_vae_kld * torch.abs(loss_KLD-C)
#     loss_vae = loss_vae.mean() # mean along the batches


df_vae[df_vae.level<2]


Total parameters 8909862
Total trainable parameters 8909862


,name,class_name,input_shape,output_shape,nb_params,level
4,encoder.0,BasicConv2d,"[(-1, 3, 256, 256)]","[(-1, 48, 256, 256)]",1440,1
42,encoder.1,ConvBlock5,"[(-1, 48, 256, 256)]","[(-1, 96, 128, 128)]",93213,1
80,encoder.2,ConvBlock5,"[(-1, 96, 128, 128)]","[(-1, 144, 64, 64)]",281034,1
118,encoder.3,ConvBlock5,"[(-1, 144, 64, 64)]","[(-1, 192, 32, 32)]",566055,1
156,encoder.4,ConvBlock5,"[(-1, 192, 32, 32)]","[(-1, 240, 16, 16)]",948276,1
194,encoder.5,ConvBlock5,"[(-1, 240, 16, 16)]","[(-1, 288, 8, 8)]",1427697,1
232,encoder.6,ConvBlock5,"[(-1, 288, 8, 8)]","[(-1, 32, 8, 8)]",351550,1
233,mu,Linear,"[(-1, 2048)]","[(-1, 256)]",524544,0
234,logvar,Linear,"[(-1, 2048)]","[(-1, 256)]",524544,0
235,z,Linear,"[(-1, 256)]","[(-1, 2048)]",526336,0


In [15]:
with TorchSummarizeDf(mdnrnn) as tdf: 
    pi, mu, sigma, hidden_state = mdnrnn.forward(z.unsqueeze(1).repeat((1,2,1)), action.repeat((1,2)))
    z_next = mdnrnn.sample(pi, mu, sigma)
#     loss_mdn = mdnrnn.rnn_loss(z, pi, mu, sigma).mean()
    df_mdnrnn = tdf.make_df()
    
df_mdnrnn

Total parameters 1778688
Total trainable parameters 1778688


,name,class_name,input_shape,output_shape,nb_params,level
1,rnn,LSTM,"[[(-1, 2, 266)], [[(-1, 1, 128)], [(-1, 1, 128...","[[(-1, 2, 128)], [[(-1, 1, 128)], [(-1, 1, 128...",202752,0
2,ln1,Linear,"[(-1, 128), (-1, 128)]","[(-1, 128), (-1, 128)]",16512,0
3,ln2,Linear,"[(-1, 128), (-1, 128)]","[(-1, 640), (-1, 640)]",82560,0
4,mdn,Linear,"[(-1, 640), (-1, 640)]","[(-1, 2304), (-1, 2304)]",1476864,0


In [16]:
#     loss = loss_mdn + gamma_vae * loss_vae
with TorchSummarizeDf(finv) as tdf:
    action_pred = finv(z.repeat((1,2,1)), z_next)
        
    df_finv = tdf.make_df()
    

df_finv

Total parameters 199690
Total trainable parameters 199690


,name,class_name,input_shape,output_shape,nb_params,level
1,ln1,Linear,"[(-1, 2, 512)]","[(-1, 2, 256)]",131328,0
2,ln2,Linear,"[(-1, 2, 256)]","[(-1, 2, 256)]",65792,0
3,ln3,Linear,"[(-1, 2, 256)]","[(-1, 2, 10)]",2570,0


In [ ]:
with TorchSummarizeDf(world_model) as tdf:
    z_next = world_model(gpu_img, action)
    z_next.shape
    df_world_model = tdf.make_df()
df_world_model[df_world_model.level<2]

torch.Size([1, 256]) torch.Size([1, 1])


In [ ]:
del img, action, gpu_img, x, mu, z

# Train helpers

# Train

In [ ]:
from deep_rl.utils import Config
from deep_rl.utils.logger import get_logger, get_default_log_dir

from deep_rl.network.network_heads import CategoricalActorCriticNet, QuantileNet, OptionCriticNet, DeterministicActorCriticNet, GaussianActorCriticNet
from deep_rl.network.network_bodies import FCBody

from deep_rl.agent.PPO_agent import PPOAgent
from deep_rl.component.task import ParallelizedTask, BaseTask
from deep_rl.utils.misc import run_episodes, run_iterations

In [ ]:
# Env
from world_models_sonic.custom_envs.env import make_env

class SonicDeepRL(BaseTask):
    def __init__(self, name='sonic256', max_steps=6000, log_dir=None):
        BaseTask.__init__(self)
        self.name = name
        self.env = make_env(self.name)
        self.env._max_episode_steps = max_steps
        self.action_dim = self.env.action_space.n
        self.state_dim = self.env.observation_space.shape
        self.env = self.set_monitor(self.env, log_dir)
        


In [ ]:
import gym
from gym import spaces

# def np_sigmoid(x):
#     return 1.0/(1.0+np.exp(-x))

# class Action2Int(gym.ActionWrapper):
#     def action(self, action):
#         return np_sigmoid(action).round(0).astype(int)
    


class WorldModelWrapper(gym.Wrapper):
    def __init__(self, env, world_model):
        """Uses the world model to give next latent state as observation."""
        super().__init__(env)
        self.render_humans = True
        self.i_rs = 0
        self.world_model = world_model
        old_obs_space = env.observation_space
        self.observation_space = spaces.Box(low=-1000, high=1000,
                                            shape=(world_model.mdnrnn.z_dim,), dtype=np.float32)
        
    def step(self, action):
        action = action.round(0).astype(int)
        
        observation, reward, done, info = self.env.step(action)
        
        action = torch.from_numpy(np.array(action)).cuda().unsqueeze(0).unsqueeze(0)
        observation = torch.from_numpy(observation).cuda().unsqueeze(0).transpose(1, 3)
        
        observation = self.world_model.forward(observation, action).squeeze(0).cpu().data.numpy()
        if self.render_humans:
            self.render()
        return observation, reward, done, info
    
    def reset(self):
        action = torch.from_numpy(np.array(self.env.action_space.sample())).cuda().unsqueeze(0).unsqueeze(0)
        observation = torch.from_numpy(self.env.reset()).cuda().unsqueeze(0).transpose(1, 3)
        observation = self.world_model.forward(observation, action).squeeze(0).cpu().data.numpy()
        if self.render_humans:
            self.render()
        return observation

class SonicWorldModelDeepRL(BaseTask):
    def __init__(self, name='sonic256', max_steps=4500, log_dir=None, world_model_func=None):
        BaseTask.__init__(self)
        self.name = name
        self.world_model = world_model_func()
        self.env = WorldModelWrapper(make_env(self.name), self.world_model)
        self.env._max_episode_steps = max_steps
        self.action_dim = self.env.action_space.n
        self.state_dim = self.env.observation_space.shape[0]
        self.env = self.set_monitor(self.env, log_dir)

In [ ]:
# task_fn = lambda log_dir: SonicWorldModelDeepRL('sonic256', max_steps=200, log_dir=log_dir, world_model_func=lambda :world_model)
# task = task_fn(log_dir)
# try:
#     task.env.reset()
#     task.env.render()
#     for i in tqdm(range(100)):
#         action = task.env.action_space.sample()
#         task.env.step(action)
#         task.env.unwrapped.step(action)
#         task.env.render()
# except:
#     task.env.unwrapped.close()
#     raise
# else:
#     task.env.unwrapped.close()

In [ ]:
log_dir = get_default_log_dir(NAME)
log_dir
task_fn = lambda log_dir: SonicWorldModelDeepRL('sonic256', max_steps=200, log_dir=log_dir, world_model_func=lambda :world_model)

config = Config()

config.num_workers = 1
config.task_fn = lambda: ParallelizedTask(task_fn, config.num_workers, single_process=config.num_workers==1)
# config.task_fn = lambda: task_fn(log_dir)
config.optimizer_fn = lambda params: torch.optim.RMSprop(params, 1e-3)
# FIXME a gaussian is not quite right for this, it should be a beta dist from 0 to 1 or a catagorical on filtered actions
# config.network_fn = lambda state_dim, action_dim: GaussianActorCriticNet(
#     state_dim, action_dim, actor_body=FCBody(state_dim),
#     critic_body=FCBody(state_dim), gpu=-1)
# config.network_fn = lambda state_dim, action_dim: GaussianActorCriticNet(
#     state_dim, action_dim, actor_body=FCBody(state_dim),
#     critic_body=FCBody(state_dim), gpu=-1)
config.network_fn = lambda state_dim, action_dim: CategoricalActorCriticNet(
        state_dim, action_dim, FCBody(state_dim), gpu=-1)
config.discount = 0.99
config.logger = get_logger(NAME, level=20)
config.use_gae = True
config.gae_tau = 0.95
config.entropy_weight = 0.01
config.gradient_clip = 0.5
config.rollout_length = 128
config.optimization_epochs = 10
config.num_mini_batches = 4
config.ppo_ratio_clip = 0.2
config.iteration_log_interval = 1
agent=PPOAgent(config)
try:
    run_iterations(agent)
except:
    agent.task.tasks[0].env.close()
    raise

In [ ]:

task = task_fn(log_dir)
try:
    task.env.reset()
    task.env.render()
    for i in tqdm(range(100)):
        action = task.env.action_space.sample()
        task.env.step(action)
        task.env.unwrapped.step(action)
        task.env.render()
except:
    task.env.unwrapped.close()
    raise

In [ ]:
task.env.unwrapped.close()

In [ ]:
env = make_env('sonic256')
env.reset()
env.render()
for i in tqdm(range(1000)):
    action = env.action_space.sample()
    env.step(action)
    env.render()

In [ ]:
env.close()